In [24]:
#create dataset
%%bigquery
CREATE SCHEMA IF NOT EXISTS airport_dataset
OPTIONS(
 location="us");

#create Table
CREATE OR REPLACE EXTERNAL TABLE airport_dataset.airports
OPTIONS (
  format = 'CSV',
  uris = ['gs://labs.roitraining.com/data-to-ai-workshop/airports.csv'],
  skip_leading_rows = 1,
  allow_jagged_rows = false,
  allow_quoted_newlines = false,
  field_delimiter = ',',
  max_bad_records = 10
)

Query is running:   0%|          |

""


In [6]:
# @title First the function that returns the weather forecast from the NWS using Lat/Long point


import requests
from typing import Optional, List, Dict


def get_extended_weather_forecast(lat: float, lon: float) -> Optional[List[Dict[str, str]]]:
   """
   Fetch the extended weather forecast from the U.S. National Weather Service (NWS) API
   based on a given latitude and longitude.


   Args:
       lat (float): Latitude of the location (e.g., 38.8977).
       lon (float): Longitude of the location (e.g., -77.0365).


   Returns:
       Optional[List[Dict[str, str]]]: A list of forecast dictionaries for each time period,
       each containing:
           - 'name': Name of the forecast period (e.g., "Today", "Tonight")
           - 'startTime': ISO timestamp for the start of the forecast period
           - 'temperature': Temperature value
           - 'temperatureUnit': Temperature unit (e.g., "F" or "C")
           - 'windSpeed': Wind speed description
           - 'windDirection': Wind direction (e.g., "NW")
           - 'shortForecast': Short summary (e.g., "Partly Sunny")
           - 'detailedForecast': Full text forecast


       Returns None if data is unavailable or an error occurs.
   """
   headers = {
       'User-Agent': 'MyWeatherApp (glen.sanford@cdwg.com)',  # Replace with your actual email
       'Accept': 'application/geo+json'
   }


   # Step 1: Get metadata to find forecast URL
   points_url = f"https://api.weather.gov/points/{lat},{lon}"
   response = requests.get(points_url, headers=headers)


   if response.status_code != 200:
       print(f"Error fetching data from points endpoint: {response.status_code}")
       return None


   points_data = response.json()
   forecast_url = points_data['properties'].get('forecast')
   if not forecast_url:
       print("Forecast URL not found in response.")
       return None


   # Step 2: Fetch the forecast data
   forecast_response = requests.get(forecast_url, headers=headers)
   if forecast_response.status_code != 200:
       print(f"Error fetching forecast: {forecast_response.status_code}")
       return None


   forecast_data = forecast_response.json()
   periods = forecast_data['properties'].get('periods', [])


   if not periods:
       print("No forecast periods found in response.")
       return None


   # Return the full extended forecast
   extended_forecast = []
   for period in periods:
       extended_forecast.append({
           'name': period['name'],
           'startTime': period['startTime'],
           'temperature': str(period['temperature']),
           'temperatureUnit': period['temperatureUnit'],
           'windSpeed': period['windSpeed'],
           'windDirection': period['windDirection'],
           'shortForecast': period['shortForecast'],
           'detailedForecast': period['detailedForecast']
       })


   return extended_forecast





In [22]:
import pandas as pd
from google.cloud import bigquery
import time
import vertexai
from vertexai.generative_models import GenerativeModel

# --- Vertex AI Gemini API Setup ---
try:
    # Initialize Vertex AI - Project and location are inferred in Colab Enterprise.
    vertexai.init()

    # Load the Gemini Pro model
    gemini_model = GenerativeModel("gemini-2.0-flash")
    print("Vertex AI Gemini API configured successfully.")
except Exception as e:
    print(f"ERROR: Failed to configure Vertex AI. {e}")
    gemini_model = None
# --- End Vertex AI Gemini API Setup ---


# Filter for large US airports, as the NWS API only supports US locations
large_us_airports_df = airports_df[(airports_df['iso_country'] == 'US') & (airports_df['type'] == 'large_airport')].copy()

alerts = []

# Proceed only if Gemini model is available
if gemini_model:
    # Iterate over large airports to get weather and generate alerts
    for index, airport in large_us_airports_df.iterrows():
        lat = airport['latitude_deg']
        lon = airport['longitude_deg']

        # Add a print statement to show progress
        print(f"Fetching weather for {airport['name']}...")

        forecast = get_extended_weather_forecast(lat, lon)
        time.sleep(1) # Add a small delay to avoid hitting API rate limits

        if forecast:
            for period in forecast:
                # Define alert conditions (e.g., thunderstorms, high wind, snow)
                if 'thunderstorm' in period['shortForecast'].lower() or 'snow' in period['shortForecast'].lower() or 'wind' in period['windSpeed'].lower():

                    print(f"  -> Found condition: {period['shortForecast']}. Generating Gemini alert...")

                    # Construct a prompt for Gemini
                    prompt = f"""
                    You are an expert aviation operations assistant. Your task is to write a concise, clear operational alert for an airport based on the provided weather forecast.

                    The alert should be a single sentence and focus on potential impacts to flights (e.g., delays, cancellations, turbulence, ground stops). Do not use introductory phrases like "The alert is:". Be professional and direct.

                    Airport: {airport['name']}
                    Time: {period['name']}
                    Forecast: {period['shortForecast']}
                    Temperature: {period['temperature']} {period['temperatureUnit']}
                    Wind: {period['windSpeed']} {period['windDirection']}

                    Generate the one-sentence passenger alert:
                    """

                    try:
                        print ("   -> Gernerating Alert with Gemini...")
                        # Generate the alert using Gemini
                        response = gemini_model.generate_content(prompt)
                        gemini_alert = response.text.strip()
                    except Exception as e:
                        print(f"    !! Gemini API failed: {e}")
                        gemini_alert = "Alert generation failed."


                    alerts.append({
                        'airport_name': airport['name'],
                        'latitude': lat,
                        'longitude': lon,
                        'forecast_time': period['startTime'],
                        'weather_condition': period['shortForecast'],
                        'gemini_generated_alert': gemini_alert
                    })

    # Create a DataFrame from the alerts
    alerts_df = pd.DataFrame(alerts)

    if not alerts_df.empty:
        # Define BigQuery table details
        project_id = 'qwiklabs-gcp-02-e82d432d54e9'
        dataset_id = 'airport_dataset'
        table_id = 'weather_alerts'
        table_ref = f"{project_id}.{dataset_id}.{table_id}"

        # Store the alerts in a BigQuery table
        print(f'\nLoading {len(alerts_df)} alerts into {table_ref}...')
        alerts_df.to_gbq(table_ref, project_id=project_id, if_exists='replace')

        print("\nSuccessfully created and populated the weather_alerts table with Gemini alerts.")
        print("Here is a preview of the new alerts data:")
        display(alerts_df.head())
    else:
        print("No weather conditions meeting the alert criteria were found.")
else:
    print("\nSkipping alert generation because the Gemini API is not configured.")

Vertex AI Gemini API configured successfully.
Fetching weather for Albuquerque International Sunport...
  -> Found condition: Mostly Sunny then Chance Showers And Thunderstorms. Generating Gemini alert...
   -> Gernerating Alert with Gemini...
  -> Found condition: Chance Showers And Thunderstorms then Partly Cloudy. Generating Gemini alert...
   -> Gernerating Alert with Gemini...
  -> Found condition: Mostly Sunny then Slight Chance Showers And Thunderstorms. Generating Gemini alert...
   -> Gernerating Alert with Gemini...
  -> Found condition: Slight Chance Showers And Thunderstorms. Generating Gemini alert...
   -> Gernerating Alert with Gemini...
  -> Found condition: Slight Chance Rain Showers then Chance Showers And Thunderstorms. Generating Gemini alert...
   -> Gernerating Alert with Gemini...
  -> Found condition: Chance Showers And Thunderstorms. Generating Gemini alert...
   -> Gernerating Alert with Gemini...
  -> Found condition: Partly Sunny then Chance Showers And Thun

/tmp/ipython-input-3066324133.py:92: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  alerts_df.to_gbq(table_ref, project_id=project_id, if_exists='replace')
100%|██████████| 1/1 [00:00<00:00, 6678.83it/s]


Successfully created and populated the weather_alerts table with Gemini alerts.
Here is a preview of the new alerts data:


,airport_name,latitude,longitude,forecast_time,weather_condition,gemini_generated_alert
0,Albuquerque International Sunport,35.039976,-106.608925,2025-08-29T06:00:00-06:00,Mostly Sunny then Chance Showers And Thunderst...,Afternoon thunderstorms may cause flight delay...
1,Albuquerque International Sunport,35.039976,-106.608925,2025-08-29T18:00:00-06:00,Chance Showers And Thunderstorms then Partly C...,Possible flight delays and disruptions due to ...
2,Albuquerque International Sunport,35.039976,-106.608925,2025-08-30T06:00:00-06:00,Mostly Sunny then Slight Chance Showers And Th...,Possible thunderstorm activity may cause fligh...
3,Albuquerque International Sunport,35.039976,-106.608925,2025-08-30T18:00:00-06:00,Slight Chance Showers And Thunderstorms,Possible flight delays due to thunderstorms an...
4,Albuquerque International Sunport,35.039976,-106.608925,2025-08-31T06:00:00-06:00,Slight Chance Rain Showers then Chance Showers...,Chance of thunderstorms Sunday may cause fligh...
